In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
filepath = 'data/rus_eng_small.txt'

In [7]:
from tqdm import tqdm_notebook
import string
import re
import random
import time
import math

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

if torch.cuda.is_available():
    print('CUDA')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA


In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    """
    Класс реализует разбор предложений на слова и построение индекса слов
    """
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.n_words = len(self.index2word)

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
!tail {filepath}

она экономически независима от своих родителей .	she is economically independent of her parents .
она направила наши усилия в нужном направлении .	she steered our efforts in the right direction .
на этом заводе производят телевизоры .	they are manufacturing tv sets in this factory .
она всегда жалуется из за моей маленькой зарплаты .	she is always complaining about my small salary .
они стоят там и едят чипсы .	they are standing there and eating potato chips .
мне не удаётся скомпилировать эту программу .	i m having some problems compiling this software .
она улыбнулась в ответ на его нежный взгляд .	she smiled in response to his affectionate glance .
она проводит каждое воскресенье со своей бабушкой .	she spends time with her grandmother every sunday .
каждую субботу после обеда она играет в теннис .	she spends every saturday afternoon playing tennis .
после аварии она перестала бывать на людях .	she stopped appearing in public after her accident .


In [14]:
def readLangs():
    """
    Загрузка параллельного корпуса
    """
    print("Loading dataset...")
    pairs = []
    # Read the file and split into lines
    with open(filepath) as f:
        for l in tqdm_notebook(f):
            pair = [s for s in l.rstrip('\n').split('\t')]
            pairs.append(pair)

    return Lang('rus'), Lang('eng'), pairs

In [15]:
def prepareData():
    """
    Чтение корпуса и инициализация словарей
    """
    input_lang, output_lang, pairs = readLangs()
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData()
MAX_LENGTH = 10
print(random.choice(pairs))

Loading dataset...



Read 19288 sentence pairs
Counting words...
Counted words:
rus 8960
eng 3914
['мы пытаемся работать .', 'we re trying to work .']


In [17]:
def indexesFromSentence(lang, sentence):
    """
    Предложение в последовательность индексов слов
    """
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    """
    Предложение в pytorch тензор
    """
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    """
    Пара параллельных предложений во входной и целевой тензоры
    """
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [18]:
class EncoderRNN(nn.Module):
    """
    Модуль рекуррентного энкодера входного предложения
    """
    def __init__(self, input_size, nlayers, hidden_size):
        """
        :param input_size - размер словаря входных текстов
        :param nlayers - число рекуррентных слоев
        :param hidden_size - размер вектора состояния рекуррентного слоя
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers)

    def forward(self, x, hidden):
        """
        :param x: [T,B] - входной тензор размерностью timestep на batch
        :param hidden - состояние рнн слоя
        :return output: [T,B,C], hidden - С=hidden_size, возвращается только последний hidden state
        """
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def initHidden(self):
        """Инициализируем состояние LSTM нулями"""
        return torch.zeros(self.nlayers, 1, self.hidden_size, device=device), \
               torch.zeros(self.nlayers, 1, self.hidden_size, device=device)

In [19]:
class DecoderRNN(nn.Module):
    """
    Модуль рекуррентного декодера
    """
    def __init__(self, hidden_size, nlayers, output_size):
        """
        :param hidden_size - размер вектора состояния рекуррентного слоя
        :param nlayers - число рекуррентных слоев
        :param output_size - размер словаря выходных текстов
        """
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x, hidden, _):
        """
        :param x: [T,B] - входной тензор размерностью timestep на batch
        :param hidden - состояние рнн слоя
        :return output: [T,B,C], hidden - С=hidden_size, возвращается только последний hidden state
        """
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        logits = self.out(output)
        output = self.softmax(logits)
        return output, hidden, None

    def initHidden(self):
        """Инициализируем состояние LSTM нулями"""
        return torch.zeros(self.nlayers, 1, self.hidden_size, device=device), \
               torch.zeros(self.nlayers, 1, self.hidden_size, device=device)

In [20]:
class AttnDecoder(nn.Module):
    """
    Декодер с механизмом внимания dot-product attention
    T1 - шаги входной последовательности
    T2 - шаги выходной последовательности
    """
    def __init__(self, hidden_size, nlayers, output_size):
        """
        :param hidden_size - размер вектора состояния рекуррентного слоя
        :param nlayers - число рекуррентных слоев
        :param output_size - размер словаря выходных текстов
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.nlayers = nlayers
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.rnn = nn.LSTM(self.hidden_size, self.hidden_size, nlayers)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        """
        :param x: [T2,B] - входной тензор с предыдущими словами размерностью timestep на batch
        :param hidden [1, B, C] - состояние рнн слоя
        :param encoder_outputs [T1,B,C] - выходы всех шагов энкодера
        :return output: [T2,B,C], hidden - С=hidden_size, возвращается только последний hidden state
        """
        embedded = self.embedding(x) # [T2,B,C]
        encoder_outputs = encoder_outputs.transpose(0, 1) # [B,T1,C]
        outputs = []
        
        for i in range(x.size(0)):
            h_t = hidden[0].transpose(0, 1).transpose(1, -1) # [B, C, 1]

            context, attn_weights = self.attention(encoder_outputs, h_t)
        
            e = embedded[i].unsqueeze(0)
            output = torch.cat((e, context), dim=-1) # [[B,1,C];[B,1,C]] -> [B,1,C*2]
            output = self.attn_combine(output).transpose(0,1) # [B,1,C*2] * [C*2,C] = [B,1,C] -transpose-> [1,B,C]
            output = F.relu(output)
            output, hidden = self.rnn(output, hidden)

            logits = self.out(output)
            output = F.log_softmax(logits, dim=-1)
            outputs.append(output)
            
        output = torch.cat(outputs, dim=0)
        return output, hidden, attn_weights

    def attention(self, encoder_outputs, h):
        # bmm is the batch matrix multiplication. 
        # The first dimension of arguments is the batch dimension and should be the same
        # [B,T1,C]x[B,C,1] -> B [T1,C]x[C,1] = [B,T1,1]
        attn_weights = F.softmax(torch.bmm(encoder_outputs, h), dim=-2) 

        # Для каждой позиции выхода веса содержат значения для всех позиций входа T1
        # В таком случае взвешенная сумма может быть посчитана за одно матричное умножение
        # [B,1,T1]x[B,T1,C] -> B [1,T1]x[T1,C] = [B, 1, C]
        context = torch.bmm(attn_weights.transpose(1,-1), encoder_outputs)
        # на выходе имеем вектор контекста размерностью C для каждой позиции выхода t2
        return context, attn_weights

In [21]:
def train(input_tensor, target_tensor, encoder, decoder, 
          optimizer,  criterion, max_length=MAX_LENGTH):
    """
    Прогоняем forward и backward pass на одном батче
    """
    encoder_hidden = encoder.initHidden()

    optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden # STATE TRANSFER
    
    if teacher_forcing_ratio == 1.: # use fast parallel mode
        decoder_input = torch.cat([decoder_input, target_tensor[:-1]], dim=0)
        decoder_outputs, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        decoder_outputs = decoder_outputs.squeeze()
        loss += criterion(decoder_outputs, target_tensor.squeeze())
        loss_value = loss.item()
    else:
        for di in range(target_length):
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_output = decoder_output.view(decoder_output.size(0), -1)
            loss += criterion(decoder_output, target_tensor[di])
            if use_teacher_forcing:
                decoder_input = target_tensor[di].unsqueeze(0)  # Teacher forcing
            else:
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.detach()
        loss_value =  loss.item() / target_length

    loss.backward()

    optimizer.step()

    return loss_value

In [22]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [23]:
def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [24]:
def run_training(encoder, decoder, n_iters, print_every=1000, plot_every=100):
    """
    Цикл тренировки
    """
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    for i in range(1, n_iters + 1):
        training_pair = training_pairs[i - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        if i > n_iters / 2:
            teacher_forcing_ratio = (n_iters - i) / n_iters
        loss = train(input_tensor, target_tensor, encoder, decoder, optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            dt = time_since(start, i / n_iters)
            print('%s (%d %d%%) %.4f' % (dt, i, i / n_iters * 100, print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses);

In [25]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """
    Цикл тестирования
    """
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        
        encoder_hidden = encoder.initHidden()
        encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden # STATE TRANSFER
    
        decoded_words = []
        decoder_attentions = []
        for di in range(max_length):
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions.append(decoder_attention)
            decoder_output = decoder_output.view(decoder_output.size(0), -1)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()
            if topi.item() == EOS_token:
                break
            word = output_lang.index2word[topi.item()]
            decoded_words.append(word)

        return decoded_words, decoder_attentions

In [26]:
def evaluate_randomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [27]:
n_iters=40000
training_pairs = [tensorsFromPair(random.choice(pairs)) for i in tqdm_notebook(range(n_iters), desc='prepare set')]

In [28]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, 1, hidden_size).to(device)
decoder = AttnDecoder(hidden_size, 1, output_lang.n_words).to(device)

In [29]:
teacher_forcing_ratio = 1.
learning_rate=0.001
optimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=learning_rate)
criterion = nn.NLLLoss()

In [30]:
print('train attention seq2seq')
run_training(encoder, decoder, n_iters, print_every=2000)

train attention seq2seq
0m 45s (- 14m 18s) (2000 5%) 2.9839
1m 14s (- 11m 10s) (4000 10%) 2.4279
1m 52s (- 10m 40s) (6000 15%) 2.2564
2m 37s (- 10m 31s) (8000 20%) 2.0926
3m 24s (- 10m 14s) (10000 25%) 2.0371
3m 56s (- 9m 12s) (12000 30%) 1.8566
4m 39s (- 8m 39s) (14000 35%) 1.8152
5m 16s (- 7m 55s) (16000 40%) 1.7765
5m 55s (- 7m 14s) (18000 45%) 1.6435
6m 23s (- 6m 23s) (20000 50%) 1.5987
7m 5s (- 5m 48s) (22000 55%) 1.5804
7m 56s (- 5m 17s) (24000 60%) 1.5797
8m 42s (- 4m 41s) (26000 65%) 1.4962
9m 32s (- 4m 5s) (28000 70%) 1.4456
10m 13s (- 3m 24s) (30000 75%) 1.4167
11m 0s (- 2m 45s) (32000 80%) 1.4181
11m 49s (- 2m 5s) (34000 85%) 1.3813
12m 22s (- 1m 22s) (36000 90%) 1.3488
12m 55s (- 0m 40s) (38000 95%) 1.3018
13m 35s (- 0m 0s) (40000 100%) 1.3008


In [ ]:
print('train attention seq2seq')
run_training(encoder, decoder, n_iters, print_every=2000)

train attention seq2seq
0m 41s (- 13m 12s) (2000 5%) 1.2037
1m 10s (- 10m 32s) (4000 10%) 1.1937
1m 37s (- 9m 14s) (6000 15%) 1.1577
2m 5s (- 8m 21s) (8000 20%) 1.1137
2m 32s (- 7m 38s) (10000 25%) 1.1449


In [ ]:
evaluate_randomly(encoder, decoder)

In [ ]:
# давайте посмотрим на что похож attention
text = "я очень осторожна ."
output_words, attentions = evaluate(encoder, decoder, text)
print(attentions[0].size(),attentions[1].size())
attentions = torch.cat(attentions).squeeze().cpu()
plt.matshow(attentions.numpy());

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    """
    Выведем attention более детально, подписав токены в позициях входной и выходной последовательностей
    """
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split() +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence)
    attentions = torch.cat(attentions).squeeze().cpu()
    print(attentions.size())
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [ ]:
evaluateAndShowAttention("я очень осторожна .")

In [ ]:
evaluateAndShowAttention("я бы выпил .")